In [1]:
# ! pip install ftfy regex tqdm
# ! pip install git+https://github.com/openai/CLIP.git

In [2]:
import numpy as np
import torch
from pkg_resources import packaging

print("Torch version:", torch.__version__)

Torch version: 2.1.1+cu121


In [3]:
import clip

clip.available_models()


/home/hoemam/anaconda3/envs/clip_prefix_caption/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


['RN50',
 'RN101',
 'RN50x4',
 'RN50x16',
 'RN50x64',
 'ViT-B/32',
 'ViT-B/16',
 'ViT-L/14',
 'ViT-L/14@336px']

In [4]:
model, preprocess = clip.load("ViT-B/32")
model.eval()
input_resolution = model.visual.input_resolution
context_length = model.context_length
vocab_size = model.vocab_size

print("Model parameters:", f"{np.sum([int(np.prod(p.shape)) for p in model.parameters()]):,}")
print("Input resolution:", input_resolution)
print("Context length:", context_length)
print("Vocab size:", vocab_size)

Model parameters: 151,277,313
Input resolution: 224
Context length: 77
Vocab size: 49408


In [5]:
from pyexpat import features
import copy
import math
from sys import prefix
import numpy as np
import pytorch_lightning as pl
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchmetrics
import copy

from transformers import CLIPModel, AutoConfig, AutoModel

In [6]:
torch.cuda.is_available()

True

In [7]:
import os

import pandas as pd
import torch
from PIL import Image
from sklearn.preprocessing import MultiLabelBinarizer
from torch.utils.data import Dataset
from transformers import CLIPTokenizer, CLIPProcessor, AutoTokenizer


class HatefulMemesDataset(Dataset):
    def __init__(self, root_folder, image_folder, split='train', labels='original', image_size=224):
        super(HatefulMemesDataset, self).__init__()
        self.root_folder = root_folder
        self.image_folder = image_folder
        self.split = split
        self.labels = labels
        self.image_size = image_size
        self.info_file = f"{self.split}.csv"

        print("data here: ", self.info_file)
        self.df = pd.read_csv(self.info_file)
        # float_cols = self.df.select_dtypes(float).columns
        # self.df[float_cols] = self.df[float_cols].fillna(-1).astype('Int64')

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        item = {}
        image_fn = row['img'].split('/')[1]
        item['image'] = Image.open(f"{self.image_folder}/{image_fn}").convert('RGB').resize((self.image_size, self.image_size))
        item['text'] = row['text']
        item['label'] = row['label']
        item['idx_meme'] = row['id']
        return item

In [8]:
m = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

In [9]:
# CLIPModel
default_param = "openai/clip-vit-large-patch14"
my_clip = CLIPModel.from_pretrained(default_param)

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["bos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["eos_token_id"]` will be overriden.


In [10]:
class CustomCollator(object):

    def __init__(self):
        pre_trained_model = 'openai/clip-vit-base-patch32'
        self.image_processor = CLIPProcessor.from_pretrained(pre_trained_model)
        self.text_processor = CLIPTokenizer.from_pretrained(pre_trained_model)

    def __call__(self, batch):
        pixel_values = self.image_processor(images=[item['image'] for item in batch], return_tensors="pt")['pixel_values']
        text_output = self.text_processor([item['text'] for item in batch], padding=True, return_tensors="pt", truncation=True)
        labels = torch.LongTensor([item['label'] for item in batch])
        idx_memes = torch.LongTensor([item['idx_meme'] for item in batch])

        batch_new = {}
        batch_new['pixel_values'] = pixel_values,
        batch_new['input_ids'] = text_output['input_ids']
        batch_new['attention_mask'] = text_output['attention_mask']

        batch_new['labels'] = labels
        batch_new['idx_memes'] = idx_memes

        return batch_new

collator = CustomCollator()

In [11]:
import multiprocessing

cores = multiprocessing.cpu_count() # Count the number of cores in a computer
cores
multilingual_tokenizer_path = 'none'
fine_grained_labels = []
compute_fine_grained_metrics = True

# Load Datasets

In [12]:
labels = "original"
image_folder = '../data/img'
image_size = 224

dataset_train = HatefulMemesDataset(root_folder='.', image_folder=image_folder, split='train',
            labels=labels, image_size=image_size)

dataset_val = HatefulMemesDataset(root_folder='.', image_folder=image_folder, split='dev',
            labels=labels, image_size=image_size)

data here:  train.csv
data here:  dev.csv


In [13]:
dataset_train[0]

{'image': <PIL.Image.Image image mode=RGB size=224x224>,
 'text': 'its their character not their color that matters',
 'label': 0,
 'idx_meme': 42953}

In [26]:
from torch.utils.data import DataLoader

dataloader_train = DataLoader(dataset_train, batch_size=265, shuffle=True, num_workers=15, collate_fn=collator)
dataloader_val = DataLoader(dataset_val, batch_size=265, shuffle=False, num_workers=15, collate_fn=collator)

In [15]:
compute_fine_grained_metrics

True

In [16]:
dataloader_train

# Train The Model

In [17]:
class CLIPClassifier(pl.LightningModule):

    def __init__(self, args, fine_grained_labels, compute_fine_grained_metrics):
        super().__init__()
        
        # focal loss parameters
        self.alpha = 0.9
        self.gamma = 10

        # self.caption_mode = args.caption_mode
        self.use_pretrained_map = args['use_pretrained_map']
        self.num_mapping_layers = args['num_mapping_layers']
        self.map_dim = args['map_dim']
        self.fusion = args['fusion']
        self.num_pre_output_layers = args['num_pre_output_layers']
        self.lr = args['lr']
        self.weight_decay = args['weight_decay']
        self.weight_image_loss = args['weight_image_loss']
        self.weight_text_loss = args['weight_text_loss']
        self.weight_fine_grained_loss = args['weight_fine_grained_loss']
        self.weight_super_loss = args['weight_super_loss']
        self.fine_grained_labels = fine_grained_labels
        self.compute_fine_grained_metrics = compute_fine_grained_metrics

        self.acc = torchmetrics.Accuracy(task="binary")
        self.auroc = torchmetrics.AUROC(task="binary")
        # self.precision_score = torchmetrics.Precision()
        # self.recall = torchmetrics.Recall()
        # self.f1 = torchmetrics.F1Score()

        self.validation_step_outputs = []

       

        self.clip = CLIPModel.from_pretrained(args['clip_pretrained_model'])
        self.image_encoder = copy.deepcopy(self.clip.vision_model)
        self.text_encoder = copy.deepcopy(self.clip.text_model)
        self.image_map = nn.Sequential(
                copy.deepcopy(self.clip.visual_projection),
                nn.ReLU(),
                nn.Linear(self.clip.projection_dim, self.map_dim)
                )
        self.text_map = nn.Sequential(
            copy.deepcopy(self.clip.text_projection),
            nn.ReLU(),
            nn.Linear(self.clip.projection_dim, self.map_dim)
            )


        

        if args['fusion'] in ['align', 'align_shuffle']:
            pre_output_input_dim = self.map_dim
        elif args['fusion'] == 'concat':
            pre_output_input_dim = self.map_dim*2
        elif args['fusion'].startswith('cross'):
            pre_output_input_dim = self.map_dim**2
        elif args['fusion'] == 'align_concat':
            pre_output_input_dim = self.map_dim*3
        elif args['fusion'] == 'attention_m':
            self.gen_query = nn.Linear(self.map_dim, self.map_dim//4)
            self.gen_key = nn.Linear(self.map_dim, self.map_dim//4)
            self.soft = nn.Softmax(dim=1)
            pre_output_input_dim = self.map_dim*2

        pre_output_layers = [nn.Dropout(p=args['drop_probs'])]
        output_input_dim = pre_output_input_dim


        if self.num_pre_output_layers >= 1: # first pre-output layer
            pre_output_layers.extend([nn.Linear(pre_output_input_dim, self.map_dim), nn.ReLU(), nn.Dropout(p=args['drop_probs'])])
            output_input_dim = self.map_dim
        for _ in range(1, self.num_pre_output_layers): # next pre-output layers
            pre_output_layers.extend([nn.Linear(self.map_dim, self.map_dim), nn.ReLU(), nn.Dropout(p=args['drop_probs'])])

        self.pre_output = nn.Sequential(*pre_output_layers)
        self.output = nn.Linear(output_input_dim, 1)
        self.output_image = nn.Linear(output_input_dim, 1)
        self.output_text = nn.Linear(output_input_dim, 1)

        if self.weight_image_loss > 0:
            pre_output_layers = [nn.Dropout(p=args['drop_probs'])]
            for _ in range(self.num_pre_output_layers): # next pre-output layers
                pre_output_layers.extend([nn.Linear(self.map_dim, self.map_dim), nn.ReLU(), nn.Dropout(p=args['drop_probs'])])
            self.pre_output_image = nn.Sequential(*pre_output_layers)

        if self.weight_text_loss > 0:
            pre_output_layers = [nn.Dropout(p=args['drop_probs'])]
            for _ in range(self.num_pre_output_layers): # next pre-output layers
                pre_output_layers.extend([nn.Linear(self.map_dim, self.map_dim), nn.ReLU(), nn.Dropout(p=args['drop_probs'])])
            self.pre_output_text = nn.Sequential(*pre_output_layers)

        if self.fine_grained_labels:
            # if self.dataset in ['original', 'masked', 'inpainted']:
            self.output_pc1 = nn.Linear(output_input_dim, 1)
            self.output_pc2 = nn.Linear(output_input_dim, 1)
            self.output_pc3 = nn.Linear(output_input_dim, 1)
            self.output_pc4 = nn.Linear(output_input_dim, 1)
            self.output_pc5 = nn.Linear(output_input_dim, 1)
            self.output_pc6 = nn.Linear(output_input_dim, 1)
            self.output_attack1 = nn.Linear(output_input_dim, 1)
            self.output_attack2 = nn.Linear(output_input_dim, 1)
            self.output_attack3 = nn.Linear(output_input_dim, 1)
            self.output_attack4 = nn.Linear(output_input_dim, 1)
            self.output_attack5 = nn.Linear(output_input_dim, 1)
            self.output_attack6 = nn.Linear(output_input_dim, 1)
            self.output_attack7 = nn.Linear(output_input_dim, 1)
            self.output_attack8 = nn.Linear(output_input_dim, 1)
            self.outputs_fine_grained = [self.output_pc1, self.output_pc2, self.output_pc3, self.output_pc4, self.output_pc5, self.output_pc6,
                self.output_attack1, self.output_attack2, self.output_attack3, self.output_attack4, self.output_attack5, self.output_attack6, self.output_attack7, self.output_attack8]
            self.output_super = nn.Linear(15, 1)

        self.cross_entropy_loss = torch.nn.BCEWithLogitsLoss(reduction='mean')

        if args['freeze_image_encoder']:
            for _, p in self.image_encoder.named_parameters():
                p.requires_grad_(False)

        if args['freeze_text_encoder']:
            for _, p in self.text_encoder.named_parameters():
                p.requires_grad_(False)

        del self.clip
        # if self.caption_mode == 'replace_image':
        #     del self.image_encoder, self.image_map

    def forward(self, batch):

        image_features = self.image_encoder(pixel_values=batch['pixel_values'][0]).pooler_output
        image_features = self.image_map(image_features)
        text_features = self.text_encoder(input_ids=batch['input_ids'], attention_mask=batch['attention_mask']).pooler_output

        image_features = F.normalize(image_features, p=2, dim=1) # [batch_size, d]
        text_features = F.normalize(text_features, p=2, dim=1) # [batch_size, d]

        features = torch.mul(image_features, text_features)  # [batch_size, d]

        features = self.pre_output(features)
        logits = self.output(features)
        preds = (torch.sigmoid(logits) >= 0.5).long()

        return preds

    def common_step(self, batch, batch_idx, calling_function='validation'):
        image_features = self.image_encoder(pixel_values=batch['pixel_values'][0]).pooler_output
        image_features = self.image_map(image_features)

        text_features = self.text_encoder(input_ids=batch['input_ids'], attention_mask=batch['attention_mask']).pooler_output
        text_features = self.text_map(text_features)

        image_features = F.normalize(image_features, p=2, dim=1)
        text_features = F.normalize(text_features, p=2, dim=1)

        output = {}

        if self.weight_image_loss > 0:
            features_pre_output = self.pre_output_image(image_features)
            logits = self.output_image(features_pre_output).squeeze(dim=1) # [batch_size, 1]
            preds_proxy = torch.sigmoid(logits)
            preds = (preds_proxy >= 0.5).long()

            output['image_loss'] = self.cross_entropy_loss(logits, batch['labels'].float())
            output['image_accuracy'] = self.acc(preds, batch['labels'])
            output['image_auroc'] = self.auroc(preds_proxy, batch['labels'])

        if self.weight_text_loss > 0:
            features_pre_output = self.pre_output_text(text_features)
            logits = self.output_text(features_pre_output).squeeze(dim=1) # [batch_size, 1]
            preds_proxy = torch.sigmoid(logits)
            preds = (preds_proxy >= 0.5).long()

            output['text_loss'] = self.cross_entropy_loss(logits, batch['labels'].float())
            output['text_accuracy'] = self.acc(preds, batch['labels'])
            output['text_auroc'] = self.auroc(preds_proxy, batch['labels'])


        features = torch.mul(image_features, text_features)

        features_pre_output = self.pre_output(features)
        logits = self.output(features_pre_output).squeeze(dim=1) # [batch_size, 1(or)n]
        if self.fine_grained_labels and self.dataset in ['original', 'masked', 'inpainted']:
            logits_for_super = [torch.relu(logits)]
        preds_proxy = torch.sigmoid(logits)
        preds = (preds_proxy >= 0.5).long()

        output['loss'] = self.cross_entropy_loss(logits, batch['labels'].float())
        # BCE_loss = F.binary_cross_entropy_with_logits(preds.float(), batch['labels'].float(), reduction='none')
        # pt = torch.exp(-BCE_loss) # prevents nans when probability 0

      
        # output['loss'] = torch.mean(torch.Tensor(self.alpha * (1-pt)**self.gamma * BCE_loss))
        # output['loss'].requires_grad_()
        output['accuracy'] = self.acc(preds, batch['labels'])
        output['auroc'] = self.auroc(preds_proxy, batch['labels'])



        if calling_function == 'training' and self.fine_grained_labels and self.outputs_fine_grained:
            for fine_grained_label, output_fine_grained in zip(self.fine_grained_labels, self.outputs_fine_grained):
                logits = output_fine_grained(features_pre_output).squeeze(dim=1)
                logits_for_super.append(torch.relu(logits))
                preds_proxy = torch.sigmoid(logits)
                preds = (preds_proxy >= 0.5).long()
                output[f'{fine_grained_label}_loss'] = self.cross_entropy_loss(logits, batch[fine_grained_label].float())
            logits_for_super = torch.stack(logits_for_super, dim=1) # [batch_size, 15]
            logits = self.output_super(logits_for_super).squeeze(dim=1)
            preds_proxy = torch.sigmoid(logits)
            preds = (preds_proxy >= 0.5).long()
            output['super_loss'] = self.cross_entropy_loss(logits, batch['labels'].float())
            output['super_accuracy'] = self.acc(preds, batch['labels'])
            output['super_auroc'] = self.auroc(preds_proxy, batch['labels'])


        elif calling_function == 'validation' and self.fine_grained_labels and self.outputs_fine_grained:
            for fine_grained_label, output_fine_grained in zip(self.fine_grained_labels, self.outputs_fine_grained):
                logits = output_fine_grained(features_pre_output).squeeze(dim=1)
                logits_for_super.append(torch.relu(logits))
                preds_proxy = torch.sigmoid(logits)
                preds = (preds_proxy >= 0.5).long()
                output[f'{fine_grained_label}_loss'] = self.cross_entropy_loss(logits, batch[fine_grained_label].float())
                output[f'{fine_grained_label}_accuracy'] = self.acc(preds, batch[fine_grained_label])
                output[f'{fine_grained_label}_auroc'] = self.auroc(preds_proxy, batch[fine_grained_label])
                
            logits_for_super = torch.stack(logits_for_super, dim=1) # [batch_size, 15]
            logits = self.output_super(logits_for_super).squeeze(dim=1)
            preds_proxy = torch.sigmoid(logits)
            preds = (preds_proxy >= 0.5).long()
            output[f'super_loss'] = self.cross_entropy_loss(logits, batch['labels'].float())
            output[f'super_accuracy'] = self.acc(preds, batch['labels'])
            output[f'super_auroc'] = self.auroc(preds_proxy, batch['labels'])

        elif calling_function == 'visualisation-v1':
            return image_features, text_features

        elif calling_function == 'visualisation-v2':
            return features

        return output

    def training_step(self, batch, batch_idx):
        output = self.common_step(batch, batch_idx, calling_function='training')

        if self.weight_image_loss > 0:
            image_loss = output['image_loss']
        else:
            image_loss = 0

        if self.weight_text_loss > 0:
            text_loss = output['text_loss']
        else:
            text_loss = 0

        if self.fine_grained_labels and self.outputs_fine_grained:
            fine_grained_loss = 0
            for fine_grained_label in self.fine_grained_labels:
                fine_grained_loss += output[f'{fine_grained_label}_loss']
            fine_grained_loss /= len(self.fine_grained_labels)
            super_loss = output['super_loss']
        else:
            fine_grained_loss = 0.0
            super_loss = 0.0

        total_loss = output['loss'] + self.weight_image_loss * image_loss + self.weight_text_loss * text_loss + self.weight_fine_grained_loss * fine_grained_loss + self.weight_super_loss * super_loss

        self.log('train/total_loss', total_loss)
        self.log('train/loss', output['loss'])
        self.log('train/accuracy', output['accuracy'])
        self.log('train/auroc', output['auroc'])

        if self.weight_image_loss > 0:
            self.log('train/image_loss', image_loss)
        if self.weight_text_loss > 0:
            self.log('train/text_loss', text_loss)

        self.log('train/fine_grained_loss', fine_grained_loss)
        self.log('train/super_loss', super_loss)

        return total_loss

    def validation_step(self, batch, batch_idx):
        output = self.common_step(batch, batch_idx, calling_function='validation')

        if self.weight_image_loss > 0:
            image_loss = output['image_loss']
        else:
            image_loss = 0

        if self.weight_text_loss > 0:
            text_loss = output['text_loss']
        else:
            text_loss = 0

        if self.fine_grained_labels and self.outputs_fine_grained:
            fine_grained_loss = torch.mean(torch.Tensor([output[f'{fine_grained_label}_loss'] for fine_grained_label in self.fine_grained_labels]))
            super_loss = output['super_loss']
        else:
            fine_grained_loss = 0.0
            super_loss = 0.0

        total_loss = output['loss'] + self.weight_image_loss * image_loss + self.weight_text_loss * text_loss + self.weight_fine_grained_loss * fine_grained_loss + self.weight_super_loss * super_loss

        self.log(f'val/total_loss', total_loss)
        self.log(f'val/loss', output['loss'])
        self.log(f'val/accuracy', output['accuracy'])
        self.log(f'val/auroc', output['auroc'])

        if self.weight_image_loss > 0:
            self.log(f'val/image_loss', image_loss)
        if self.weight_text_loss > 0:
            self.log(f'val/text_loss', text_loss)


        # TODO include this logic if needed
        if self.fine_grained_labels and self.compute_fine_grained_metrics:
            self.log(f'val/fine_grained_loss', fine_grained_loss)
            self.log(f'val/super_loss', super_loss)

            for fine_grained_label in self.fine_grained_labels:
                self.log(f'val-fine-grained/{fine_grained_label}_accuracy', output[f'{fine_grained_label}_accuracy'])
                self.log(f'val-fine-grained/{fine_grained_label}_auroc', output[f'{fine_grained_label}_auroc'])
                # self.log(f'val-fine-grained/{fine_grained_label}_precision', output[f'{fine_grained_label}_precision'])
                # self.log(f'val-fine-grained/{fine_grained_label}_recall', output[f'{fine_grained_label}_recall'])
                # self.log(f'val-fine-grained/{fine_grained_label}_f1', output[f'{fine_grained_label}_f1'])

            self.log(f'val/super_loss', output['super_loss'])
            self.log(f'val/super_accuracy', output['super_accuracy'])
            self.log(f'val/super_auroc', output['super_auroc'])

        self.validation_step_outputs.append(total_loss)
        return total_loss

    # def on_train_epoch_end(self, validation_step_outputs):
    def on_train_epoch_end(self):
        self.acc.reset()
        self.auroc.reset()
        # self.precision_score.reset()
        # self.recall.reset()
        # self.f1.reset()

    # def on_validation_epoch_end(self, validation_step_outputs):
    def on_validation_epoch_end(self):

        self.acc.reset()
        self.auroc.reset()
        # self.precision_score.reset()
        # self.recall.reset()
        # self.f1.reset()

        epoch_average = torch.stack(self.validation_step_outputs).mean()
        self.log("validation_epoch_average", epoch_average)
        self.validation_step_outputs.clear()  # free memory

    # def test_epoch_end(self, validation_step_outputs):
    #     self.acc.reset()
    #     self.auroc.reset()
    #     self.precision_score.reset()
    #     self.recall.reset()
    #     self.f1.reset()

    def configure_optimizers(self):
        param_dicts = [
            {"params": [p for n, p in self.named_parameters() if p.requires_grad]}
            ]
        # print("what are params ", param_dicts)
        optimizer = torch.optim.AdamW(param_dicts, lr=self.lr, weight_decay=self.weight_decay)

        return optimizer


def create_model(args, fine_grained_labels):
    compute_fine_grained_metrics = True
    model = CLIPClassifier(args=args, fine_grained_labels=fine_grained_labels, compute_fine_grained_metrics = compute_fine_grained_metrics)

    return model

In [27]:
# setup params
clip_model = "openai/clip-vit-base-patch32"

# default_param = {
#     "use_pretrained_map": False,
#     "num_mapping_layers": 1,
#     "map_dim": 768,
#     "fusion": "align",
#     "num_pre_output_layers": 1,
#     "lr": 1e-4,
#     "weight_decay": 1e4,
#     "weight_image_loss": 1.0,
#     "weight_text_loss": 1.0,
#     "weight_fine_grained_loss": 1.0,
#     "weight_super_loss": 1.0,
#     "fine_grained_labels": [],
#     "clip_pretrained_model": clip_model,
#     "drop_probs": 0.1,
#     "freeze_image_encoder": True,
#     "freeze_text_encoder": True
# }

# default_param = {
#     "use_pretrained_map": False,
#     "num_mapping_layers": 1,
#     "map_dim": 768,
#     "fusion": "align",
#     "num_pre_output_layers": 1,
#     "lr": 1e-4,
#     "weight_decay": 1e4,
#     "weight_image_loss": 1.0,
#     "weight_text_loss": 1.0,
#     "weight_fine_grained_loss": 1.0,
#     "weight_super_loss": 1.0,
#     "fine_grained_labels": [],
#     "clip_pretrained_model": clip_model,
#     "drop_probs": 0.1,
#     "freeze_image_encoder": True,
#     "freeze_text_encoder": True
# }

default_param = {
    "use_pretrained_map": False,
    "num_mapping_layers": 1,
    "map_dim": 768,
    "fusion": "align",
    "num_pre_output_layers": 1,
    "lr": 1e-2,
    "weight_decay": 0,
    "weight_image_loss": 0,
    "weight_text_loss": 0,
    "weight_fine_grained_loss": 0,
    "weight_super_loss": 0,
    "fine_grained_labels": [],
    "clip_pretrained_model": clip_model,
    "drop_probs": 0.1,
    "freeze_image_encoder": True,
    "freeze_text_encoder": True
}


from pytorch_lightning import Trainer, seed_everything


model = create_model(default_param, fine_grained_labels=[])

In [28]:
model

CLIPClassifier(
  (acc): BinaryAccuracy()
  (auroc): BinaryAUROC()
  (image_encoder): CLIPVisionTransformer(
    (embeddings): CLIPVisionEmbeddings(
      (patch_embedding): Conv2d(3, 768, kernel_size=(32, 32), stride=(32, 32), bias=False)
      (position_embedding): Embedding(50, 768)
    )
    (pre_layrnorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (encoder): CLIPEncoder(
      (layers): ModuleList(
        (0-11): 12 x CLIPEncoderLayer(
          (self_attn): CLIPAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (layer_norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): CLIPMLP(
            (activation_fn): QuickGELUActivation()
            (fc1): Linear(in_f

In [20]:
# from pytorch_lightning.callbacks import ModelCheckpoint

# monitor="val/auroc"
# project="meme-v2"

# checkpoint_callback = ModelCheckpoint(dirpath='checkpoints', filename=project,  monitor=monitor, 
#                                       mode='max', verbose=True, save_weights_only=True, save_top_k=3, save_last=False)

In [29]:
from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.callbacks import ModelCheckpoint

# seed_everything(42)
model = create_model(default_param, fine_grained_labels=[])


# #TODO add GPU later
# gpus=args.gpus

max_steps = -1
gradient_clip_val = 0.1
log_every_n_steps = 5
max_epochs = 20
val_check_interval = 1.0
limit_train_batches = 1.0
limit_val_batches = 1.0

monitor="val/auroc"
project="meme-v2"


checkpoint_callback = ModelCheckpoint(dirpath='checkpoints', filename='checkpointFile',  
                                      monitor=monitor, mode='max', verbose=True, save_weights_only=True, save_top_k=3, save_last=False)

# accelerator="cpu", devices=2
# accelerator="gpu", devices=1

# mode data and model to device (GPU)

trainer = Trainer(max_epochs=max_epochs, max_steps=max_steps, gradient_clip_val=gradient_clip_val, 
        log_every_n_steps=log_every_n_steps, val_check_interval=val_check_interval, accelerator="gpu", devices=1,
        strategy="auto", callbacks=[checkpoint_callback],
        limit_train_batches=limit_train_batches, limit_val_batches=limit_val_batches,
        deterministic=True)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(limit_train_batches=1.0)` was configured so 100% of the batches per epoch will be used..
`Trainer(limit_val_batches=1.0)` was configured so 100% of the batches will be used..
`Trainer(val_check_interval=1.0)` was configured so validation will run at the end of the training epoch..


In [30]:
trainer.fit(model, train_dataloaders=dataloader_train, val_dataloaders=dataloader_val)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name               | Type                  | Params
--------------------------------------------------------------
0  | acc                | BinaryAccuracy        | 0     
1  | auroc              | BinaryAUROC           | 0     
2  | image_encoder      | CLIPVisionTransformer | 87.5 M
3  | text_encoder       | CLIPTextTransformer   | 63.2 M
4  | image_map          | Sequential            | 787 K 
5  | text_map           | Sequential            | 656 K 
6  | pre_output         | Sequential            | 590 K 
7  | output             | Linear                | 769   
8  | output_image       | Linear                | 769   
9  | output_text        | Linear                | 769   
10 | cross_entropy_loss | BCEWithLogitsLoss     | 0     
--------------------------------------------------------------
2.0 M     Trainable params
150 M     Non-trainable params
152 M     Total params
610.633   Total estimated model params size (MB)


Epoch 0: 100%|██████████| 33/33 [02:58<00:00,  0.19it/s, v_num=30]         

Epoch 0, global step 33: 'val/auroc' reached 0.67213 (best 0.67213), saving model to 'checkpoints/checkpointFile-v41.ckpt' as top 3


Epoch 1: 100%|██████████| 33/33 [02:53<00:00,  0.19it/s, v_num=30]

Epoch 1, global step 66: 'val/auroc' reached 0.68065 (best 0.68065), saving model to 'checkpoints/checkpointFile-v42.ckpt' as top 3


...

Epoch 7: 100%|██████████| 33/33 [02:45<00:00,  0.20it/s, v_num=30]

Epoch 7, global step 264: 'val/auroc' reached 0.69911 (best 0.71976), saving model to 'checkpoints/checkpointFile-v42.ckpt' as top 3


Epoch 8: 100%|██████████| 33/33 [02:42<00:00,  0.20it/s, v_num=30]

Epoch 8, global step 297: 'val/auroc' reached 0.70452 (best 0.71976), saving model to 'checkpoints/checkpointFile-v42.ckpt' as top 3


Epoch 9:   0%|          | 0/33 [00:00<?, ?it/s, v_num=30]         

In [23]:
# default_param = {
#     "use_pretrained_map": False,
#     "num_mapping_layers": 1,
#     "map_dim": 768,
#     "fusion": "align",
#     "num_pre_output_layers": 1,
#     "lr": 1e-2,
#     "weight_decay": 1e-2,
#     "weight_image_loss": 0,
#     "weight_text_loss": 0,
#     "weight_fine_grained_loss": 0,
#     "weight_super_loss": 0,
#     "fine_grained_labels": [],
#     "clip_pretrained_model": clip_model,
#     "drop_probs": 0.1,
#     "freeze_image_encoder": True,
#     "freeze_text_encoder": True
# }
# best is 0.7165 v_20

""" 
default_param = {
    "use_pretrained_map": False,
    "num_mapping_layers": 1,
    "map_dim": 768,
    "fusion": "align",
    "num_pre_output_layers": 1,
    "lr": 1e-2,
    "weight_decay": 0,
    "weight_image_loss": 0,
    "weight_text_loss": 0,
    "weight_fine_grained_loss": 0,
    "weight_super_loss": 0,
    "fine_grained_labels": [],
    "clip_pretrained_model": clip_model,
    "drop_probs": 0.1,
    "freeze_image_encoder": True,
    "freeze_text_encoder": True
}

best is 0.75155 v_22
------------------------------------------


default_param = {
    "use_pretrained_map": False,
    "num_mapping_layers": 1,
    "map_dim": 768,
    "fusion": "align",
    "num_pre_output_layers": 1,
    "lr": 1e-4,
    "weight_decay": 0,
    "weight_image_loss": 0,
    "weight_text_loss": 0,
    "weight_fine_grained_loss": 0,
    "weight_super_loss": 0,
    "fine_grained_labels": [],
    "clip_pretrained_model": clip_model,
    "drop_probs": 0.1,
    "freeze_image_encoder": True,
    "freeze_text_encoder": True
}


best is 0.69688  at v31


------------------------------------
default_param = {
    "use_pretrained_map": False,
    "num_mapping_layers": 1,
    "map_dim": 768,
    "fusion": "align",
    "num_pre_output_layers": 1,
    "lr": 1e-2,
    "weight_decay": 0,
    "weight_image_loss": 0,
    "weight_text_loss": 0,
    "weight_fine_grained_loss": 0,
    "weight_super_loss": 0,
    "fine_grained_labels": [],
    "clip_pretrained_model": clip_model,
    "drop_probs": 0.4,
    "freeze_image_encoder": True,
    "freeze_text_encoder": True
}

best is 0.73214 v33

 """


' \ndefault_param = {\n    "use_pretrained_map": False,\n    "num_mapping_layers": 1,\n    "map_dim": 768,\n    "fusion": "align",\n    "num_pre_output_layers": 1,\n    "lr": 1e-2,\n    "weight_decay": 0,\n    "weight_image_loss": 0,\n    "weight_text_loss": 0,\n    "weight_fine_grained_loss": 0,\n    "weight_super_loss": 0,\n    "fine_grained_labels": [],\n    "clip_pretrained_model": clip_model,\n    "drop_probs": 0.1,\n    "freeze_image_encoder": True,\n    "freeze_text_encoder": True\n}\n\nbest is 0.75155 v_22\n------------------------------------------\n\n\ndefault_param = {\n    "use_pretrained_map": False,\n    "num_mapping_layers": 1,\n    "map_dim": 768,\n    "fusion": "align",\n    "num_pre_output_layers": 1,\n    "lr": 1e-4,\n    "weight_decay": 0,\n    "weight_image_loss": 0,\n    "weight_text_loss": 0,\n    "weight_fine_grained_loss": 0,\n    "weight_super_loss": 0,\n    "fine_grained_labels": [],\n    "clip_pretrained_model": clip_model,\n    "drop_probs": 0.1,\n    "free

In [24]:
[{"ACMImageMulticlassExperimental0":null,"AdultImage":null,"AoaiCompletionAbuseList":false,"AoaiPromptAbuseList":false,"AoaiPromptJailbreakAbuseList":false,"BingBlockList":false,"BingCreativeAndSensitiveTopicsPattern":false,"BingCreativePattern":false,"BingMaskList":false,"BingSensitiveTopicsPattern":false,"BingSoftBlockList":false,"CodeVulnerability":[{"EntityType":"sql-injection","Val":false,"Probability":0.1},{"EntityType":"hardcoded-credentials","Val":false,"Probability":0.04}],"CombinedJailbreak":true,"Conv_MultiSeverity_HateSpeech":null,"Conv_MultiSeverity_HateSpeechScore":null,"Conv_MultiSeverity_SelfHarm":null,"Conv_MultiSeverity_SelfHarmScore":null,"Conv_MultiSeverity_Sexual":null,"Conv_MultiSeverity_SexualScore":null,"Conv_MultiSeverity_Violence":null,"Conv_MultiSeverity_ViolenceScore":null,"CreativeTextCopyright":null,"CustomerBlocklist":null,"DisinfoTopics":false,"DisinfoTopicsAbortionPattern":false,"DisinfoTopicsBLMPattern":false,"DisinfoTopicsCovidPattern":false,"DisinfoTopicsPattern":false,"DisinfoTopicsUSElectionsPattern":false,"DisinfoTopicsUkrainePattern":false,"GoreImage":null,"HateSpeech":true,"HateSpeechNeural":0.97,"HateSpeechPattern":false,"IPCodeCitations":null,"IPNLCitations":null,"ImageProvenance":null,"Img_FastMultiSeverity_HateSpeech":null,"Img_FastMultiSeverity_HateSpeechScore":null,"Img_FastMultiSeverity_SelfHarm":null,"Img_FastMultiSeverity_SelfHarmScore":null,"Img_FastMultiSeverity_Sexual":null,"Img_FastMultiSeverity_SexualScore":null,"Img_FastMultiSeverity_Violence":null,"Img_FastMultiSeverity_ViolenceScore":null,"Img_MultiSeverity_HateSpeech":null,"Img_MultiSeverity_HateSpeechScore":null,"Img_MultiSeverity_SelfHarm":null,"Img_MultiSeverity_SelfHarmScore":null,"Img_MultiSeverity_Sexual":null,"Img_MultiSeverity_SexualScore":null,"Img_MultiSeverity_Violence":null,"Img_MultiSeverity_ViolenceScore":null,"Inappropriate":0.94067967,"InterleaveTxtImg_MultiSeverity_HateSpeech":null,"InterleaveTxtImg_MultiSeverity_HateSpeechScore":null,"InterleaveTxtImg_MultiSeverity_SelfHarm":null,"InterleaveTxtImg_MultiSeverity_SelfHarmScore":null,"InterleaveTxtImg_MultiSeverity_Sexual":null,"InterleaveTxtImg_MultiSeverity_SexualScore":null,"InterleaveTxtImg_MultiSeverity_Violence":null,"InterleaveTxtImg_MultiSeverity_ViolenceScore":null,"Jailbreak":{"Val":true,"Probability":0.99966633},"JailbreakBlockList":false,"M365CompletionBlockList":false,"M365PromptBlockList":false,"Malware":false,"MalwareNeural":0.01,"MultiSeverity_HateSpeech":{"0":1,"1":0.99,"2":0.97,"3":0.88,"4":0.765,"5":0.4,"6":0.2,"7":0.1},"MultiSeverity_HateSpeechDangerous":0.1,"MultiSeverity_HateSpeechExplicit":0.2,"MultiSeverity_HateSpeechLow":0.99,"MultiSeverity_HateSpeechMature":0.4,"MultiSeverity_HateSpeechNotable":0.97,"MultiSeverity_HateSpeechOvert":0.765,"MultiSeverity_HateSpeechQuestionable":0.88,"MultiSeverity_HateSpeechScore":7,"MultiSeverity_SelfHarm":{"0":1,"1":0.9881604,"2":0.9881714,"3":0.97852737,"4":0.032060772,"5":0.017656874,"6":0.0029615331,"7":0.000039016853},"MultiSeverity_SelfHarmDangerous":0.000039016853,"MultiSeverity_SelfHarmExplicit":0.0029615331,"MultiSeverity_SelfHarmLow":0.9881604,"MultiSeverity_SelfHarmMature":0.017656874,"MultiSeverity_SelfHarmNotable":0.9881714,"MultiSeverity_SelfHarmOvert":0.032060772,"MultiSeverity_SelfHarmQuestionable":0.97852737,"MultiSeverity_SelfHarmScore":6,"MultiSeverity_Sexual":{"0":1,"1":0.9881604,"2":0.01,"3":0.088,"4":0.0765,"5":0.04,"6":0.02,"7":0.01},"MultiSeverity_SexualDangerous":0.01,"MultiSeverity_SexualExplicit":0.02,"MultiSeverity_SexualLow":0.9881604,"MultiSeverity_SexualMature":0.04,"MultiSeverity_SexualNotable":0.01,"MultiSeverity_SexualOvert":0.0765,"MultiSeverity_SexualQuestionable":0.088,"MultiSeverity_SexualScore":0,"MultiSeverity_Violence":{"0":1,"1":0.99,"2":0.01,"3":0.088,"4":0.0765,"5":0.04,"6":0.02,"7":0.01},"MultiSeverity_ViolenceDangerous":0.01,"MultiSeverity_ViolenceExplicit":0.02,"MultiSeverity_ViolenceLow":0.99,"MultiSeverity_ViolenceMature":0.04,"MultiSeverity_ViolenceNotable":0.01,"MultiSeverity_ViolenceOvert":0.0765,"MultiSeverity_ViolenceQuestionable":0.088,"MultiSeverity_ViolenceScore":4,"Offensive":0.994259,"OfficeGuardlistLowPrecision":{"Profanity":"HiPriority","Weapons":"LoPriority"},"PMNLCitations":null,"Profanity":false,"ProfanityPattern":false,"PromBlockList":false,"PromptInjection_CrossDomain":{"Val":true,"Probability":0.99966633},"PromptInjection_Jailbreak":{"Val":true,"Probability":0.99966633},"PromptPromBlockList":false,"PublicFigures":false,"PublicFiguresPattern":false,"RacyImage":null,"SelfHarm":true,"SelfHarmNeural":0.9881714,"SelfHarmPattern":false,"SensitiveTopics":["Corona"],"SensitiveTopicsProbability":0.90274304,"Sexual":false,"SexualNeural":0.01,"SexualPattern":false,"StoppingLogic":true,"SuicideHelp":0.06913209,"TextCopyright":null,"TextImg_MultiSeverity_HateSpeech":{"0":0.02,"2":0.01,"4":0.96,"6":0.01},"TextImg_MultiSeverity_HateSpeechScore":3.96,"TextToImageLowPrecision":false,"TextToImageLowPrecisionPattern":false,"Txt2CodeLowAUC":false,"Txt2CodeLowAUCObscenePattern":false,"Txt2CodeLowAUCSensitivePattern":false,"Txt2CodeLowAUCViolencePattern":false,"Txt_FastMultiSeverity_HateSpeech":{"0":1,"1":0.99,"2":0.97,"3":0.88,"4":0.765,"5":0.4,"6":0.2,"7":0.1},"Txt_FastMultiSeverity_HateSpeechDangerous":0.1,"Txt_FastMultiSeverity_HateSpeechExplicit":0.2,"Txt_FastMultiSeverity_HateSpeechLow":0.99,"Txt_FastMultiSeverity_HateSpeechMature":0.4,"Txt_FastMultiSeverity_HateSpeechNotable":0.97,"Txt_FastMultiSeverity_HateSpeechOvert":0.765,"Txt_FastMultiSeverity_HateSpeechQuestionable":0.88,"Txt_FastMultiSeverity_HateSpeechScore":7,"Txt_FastMultiSeverity_SelfHarm":{"0":1,"1":0.9881604,"2":0.9881714,"3":0.97852737,"4":0.032060772,"5":0.017656874,"6":0.0029615331,"7":0.000039016853},"Txt_FastMultiSeverity_SelfHarmDangerous":0.000039016853,"Txt_FastMultiSeverity_SelfHarmExplicit":0.0029615331,"Txt_FastMultiSeverity_SelfHarmLow":0.9881604,"Txt_FastMultiSeverity_SelfHarmMature":0.017656874,"Txt_FastMultiSeverity_SelfHarmNotable":0.9881714,"Txt_FastMultiSeverity_SelfHarmOvert":0.032060772,"Txt_FastMultiSeverity_SelfHarmQuestionable":0.97852737,"Txt_FastMultiSeverity_SelfHarmScore":6,"Txt_FastMultiSeverity_Sexual":{"0":1,"1":0.9881604,"2":0.01,"3":0.088,"4":0.0765,"5":0.04,"6":0.02,"7":0.01},"Txt_FastMultiSeverity_SexualDangerous":0.01,"Txt_FastMultiSeverity_SexualExplicit":0.02,"Txt_FastMultiSeverity_SexualLow":0.9881604,"Txt_FastMultiSeverity_SexualMature":0.04,"Txt_FastMultiSeverity_SexualNotable":0.01,"Txt_FastMultiSeverity_SexualOvert":0.0765,"Txt_FastMultiSeverity_SexualQuestionable":0.088,"Txt_FastMultiSeverity_SexualScore":0,"Txt_FastMultiSeverity_Violence":{"0":1,"1":0.99,"2":0.01,"3":0.088,"4":0.0765,"5":0.04,"6":0.02,"7":0.01},"Txt_FastMultiSeverity_ViolenceDangerous":0.01,"Txt_FastMultiSeverity_ViolenceExplicit":0.02,"Txt_FastMultiSeverity_ViolenceLow":0.99,"Txt_FastMultiSeverity_ViolenceMature":0.04,"Txt_FastMultiSeverity_ViolenceNotable":0.01,"Txt_FastMultiSeverity_ViolenceOvert":0.0765,"Txt_FastMultiSeverity_ViolenceQuestionable":0.088,"Txt_FastMultiSev

SyntaxError: EOL while scanning string literal (3603761968.py, line 1)

- Added 0 worker for dataloader and 1 device for trainer. The auroc is 0.448 and it is unchanged through different epochs.
- <the issue could be that configure_optimizer is not taking the right parameters and updating the right thing